<a href="https://colab.research.google.com/github/Paridhi1112/Agentic-AI-First-AI-Agent/blob/main/AI_Researcher_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install requests==2.32.5

In [2]:
!pip install langchain-tavily langgraph python-dotenv streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.8 MB/s eta 0:00:00


In [3]:

!pip install -q streamlit langchain-groq langchain-community tavily-python

!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.2 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [10]:
print("Installing libraries")
!pip install -q streamlit langchain-groq langchain-community tavily-python pyngrok
print("Installation complete")

Installing libraries
Installation complete


In [11]:
import os
from google.colab import userdata

# 1. Get keys from Colab
try:
    groq_key = userdata.get('GROQ_API_KEY')
    tavily_key = userdata.get('TAVILY_API_KEY')
except Exception:
    raise ValueError("Keys missing!")

# 2. Write the logic file with keys injected
logic_code = f"""
import os
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

# Inject keys
os.environ["GROQ_API_KEY"] = "{groq_key}"
os.environ["TAVILY_API_KEY"] = "{tavily_key}"

# Initialize AI
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
tavily = TavilySearchResults(max_results=3)

class ResearchState:
    def __init__(self, topic):
        self.topic = topic
        self.logs = []
        self.research_data = []
        self.analysis = ""
        self.report = ""
    def log(self, msg):
        self.logs.append(msg)

def run_pipeline(topic):
    state = ResearchState(topic)
    state.log(f"Started Job: {{topic}}")

    # 1. Researcher
    try:
        state.log("RESEARCHER: Browsing web...")
        q = llm.invoke(f"Generate 1 search query for: {{topic}}").content.strip('"')
        state.log(f"   Query: {{q}}")
        results = tavily.invoke(q)
        state.research_data = [r['content'] for r in results]
    except Exception as e:
        state.log(f"Search failed: {{e}}")

    # 2. Analyst
    try:
        state.log("ANALYST: Thinking...")
        data = "\\n".join(state.research_data)
        state.analysis = llm.invoke(f"Analyze this data for {{topic}}: {{data}}").content
    except Exception as e:
        state.analysis = "Analysis failed."

    # 3. Writer
    try:
        state.log("WRITER: Writing report...")
        state.report = llm.invoke(f"Write a markdown report on {{topic}} using: {{state.analysis}}").content
    except Exception as e:
        state.report = "Writing failed."

    return state
"""

with open("logic.py", "w") as f:
    f.write(logic_code)
print("logic.py created.")

logic.py created.


In [12]:
app_code = """
import streamlit as st
from logic import run_pipeline

st.set_page_config(page_title="Agent Squad", page_icon="⚡")
st.title("AI Research Squad")

if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Ready to research!"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if topic := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": topic})
    st.chat_message("user").write(topic)

    with st.chat_message("assistant"):
        status = st.status("Agents working...", expanded=True)
        result = run_pipeline(topic)
        for log in result.logs:
            st.write(log)
        status.update(label="Done!", state="complete", expanded=False)
        st.markdown(result.report)
        st.session_state.messages.append({"role": "assistant", "content": result.report})
"""

with open("app.py", "w") as f:
    f.write(app_code)
print("app.py created.")

app.py created.


In [7]:
!pip install -q pyngrok

In [15]:
import os
import time
from pyngrok import ngrok
from google.colab import userdata

# 1. Kill everything first (Clear the deck)
print("Killing old servers.")
ngrok.kill()
!pkill streamlit

# 2. Get Token
try:
    token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(token)
except:
    raise ValueError("NGROK_AUTH_TOKEN is missing from Secrets.")

# 3. Start Streamlit (Waiting for it to boot)
print("Starting Streamlit (Please wait 5 seconds)...")
# We bind to 0.0.0.0 to ensure it's accessible locally
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > logs.txt 2>&1 &

# Give it time to start!
time.sleep(5)

# 4. Check if it's actually running
import requests
try:
    # Ping the local server to see if it's alive
    response = requests.get("http://localhost:8501")
    if response.status_code == 200:
        print("Streamlit is running locally!")
    else:
        print("Streamlit is running but returned a weird status.")
except:
    print("ERROR: Streamlit failed to start. Printing logs below:")
    print("="*30)
    !cat logs.txt
    print("="*30)
    raise RuntimeError("Streamlit crashed. Check the logs above.")

# 5. Open the Tunnel
try:
    url = ngrok.connect(8501).public_url
    print(f"\nSUCCESS! Click here to chat:\n {url}")
except Exception as e:
    print(f" Tunnel Error: {e}")

Killing old servers.
Starting Streamlit (Please wait 5 seconds)...
Streamlit is running locally!

SUCCESS! Click here to chat:
 https://unpushed-palmira-postpyretic.ngrok-free.dev
